In [1]:
import pickle
import pandas as pd
import numpy as np
from collections import defaultdict
from settings import *
from math import sqrt

# import xgboost as xgb
from sklearn import preprocessing, neighbors, svm
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, VotingClassifier, GradientBoostingClassifier, ExtraTreesClassifier, \
  RandomForestRegressor, AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostClassifier
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression

DIR_DATA = "data"
DIR_PROCESSED = "processed"
LABEL = "rating"
NON_PREDICTORS = [LABEL, "int_rating"]#, "name","anime_id"]
CV_FOLDS = 5

In [2]:
a_file = open(DIR_PROCESSED + '/one_hot_encoded_anime.pickle', 'rb')
anime = pickle.load(a_file)
a_file.close()

# TODO: Deal with nan genre in anime

# "rating" is in both tables
anime["avg_rating"] = anime.rating
anime = anime.drop("rating", 1)

ratings = pd.read_csv(DIR_DATA + '/rating.csv')

In [3]:
anime.head()

,anime_id,name,episodes,members,Movie,Music,ONA,OVA,Special,TV,...,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri,nan,int_rating,avg_rating
0,32281,Kimi no Na wa.,1,200630,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,9.37
1,5114,Fullmetal Alchemist: Brotherhood,64,793665,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,9.26
2,28977,Gintama°,51,114262,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,9.25
3,9253,Steins;Gate,24,673572,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,9.17
4,9969,Gintama&#039;,51,151266,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,9.16


In [4]:
ratings.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [5]:
shows = pd.merge(anime, ratings)
shows.head()

,anime_id,name,episodes,members,Movie,Music,ONA,OVA,Special,TV,...,Supernatural,Thriller,Vampire,Yaoi,Yuri,nan,int_rating,avg_rating,user_id,rating
0,32281,Kimi no Na wa.,1,200630,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,9,9.37,99,5
1,32281,Kimi no Na wa.,1,200630,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,9,9.37,152,10
2,32281,Kimi no Na wa.,1,200630,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,9,9.37,244,10
3,32281,Kimi no Na wa.,1,200630,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,9,9.37,271,10
4,32281,Kimi no Na wa.,1,200630,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,9,9.37,278,-1


In [6]:
shows[shows.name == "Dragon Ball Z"].head()

,anime_id,name,episodes,members,Movie,Music,ONA,OVA,Special,TV,...,Supernatural,Thriller,Vampire,Yaoi,Yuri,nan,int_rating,avg_rating,user_id,rating
1259607,813,Dragon Ball Z,291,375662,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,8,8.32,3,10
1259608,813,Dragon Ball Z,291,375662,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,8,8.32,5,5
1259609,813,Dragon Ball Z,291,375662,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,8,8.32,7,9
1259610,813,Dragon Ball Z,291,375662,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,8,8.32,12,9
1259611,813,Dragon Ball Z,291,375662,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,8,8.32,21,8


In [7]:
 # TODO: use columns "anime_id" later
animeRatings = shows.pivot_table(index=['user_id'], columns='name', values='rating')
animeRatings.head()

name,&quot;0&quot;,"&quot;Aesop&quot; no Ohanashi yori: Ushi to Kaeru, Yokubatta Inu",&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,&quot;Bungaku Shoujo&quot; Memoire,&quot;Bungaku Shoujo&quot; Movie,&quot;Eiji&quot;,.hack//G.U. Returner,.hack//G.U. Trilogy,.hack//G.U. Trilogy: Parody Mode,.hack//Gift,...,makemagic,"on-chan, Yume Power Daibouken!",s.CRY.ed,vivi,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki,◯
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN


In [8]:
DBZRatings = animeRatings["Dragon Ball Z"]
DBZRatings.head()

user_id
1     NaN
2     NaN
3    10.0
4     NaN
5     5.0
Name: Dragon Ball Z, dtype: float64

In [9]:
similarAnimes = animeRatings.corrwith(DBZRatings)
similarAnimes = similarAnimes.dropna()
df = pd.DataFrame(similarAnimes)
df.head()

/home/altock/anaconda3/envs/py35/lib/python3.5/site-packages/numpy/lib/function_base.py:2487: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
/home/altock/anaconda3/envs/py35/lib/python3.5/site-packages/numpy/lib/function_base.py:2496: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)


,0
name,
&quot;0&quot;,0.663817
"&quot;Aesop&quot; no Ohanashi yori: Ushi to Kaeru, Yokubatta Inu",1.000000
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,0.438522
&quot;Bungaku Shoujo&quot; Memoire,0.430201
&quot;Bungaku Shoujo&quot; Movie,0.542192


In [10]:
similarAnimes.order(ascending=False)

/home/altock/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: order is deprecated, use sort_values(...)
  if __name__ == '__main__':


name
Tamagotchi! Miracle Friends                                     1.000000
3-tsu no Hanashi                                                1.000000
Ojiichan ga Kaizoku Datta Koro                                  1.000000
Itazura Tenshi Chippo-chan                                      1.000000
Itoshi no Betty Mamonogatari                                    1.000000
Oishinbo: Nichibei Kome Sensou                                  1.000000
Oishinbo                                                        1.000000
Jam                                                             1.000000
Jankenman                                                       1.000000
Okashina Hotel                                                  1.000000
Jarinko Chie (TV)                                               1.000000
Obocchama-kun                                                   1.000000
Wan Wan Chuushingura                                            1.000000
Wan Wan Celepoo Soreyuke! Tetsunoshin         

In [11]:
animeStats = shows.groupby('name').agg({'rating': [np.size, np.mean]})
animeStats.head()

rating          
                                                     size      mean
name                                                               
&quot;0&quot;                                          26  2.769231
&quot;Aesop&quot; no Ohanashi yori: Ushi to Kae...      2  0.000000
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hats...    782  5.774936
&quot;Bungaku Shoujo&quot; Memoire                    809  6.155748
&quot;Bungaku Shoujo&quot; Movie                     1535  6.457980

In [12]:
# TODO: Use avgRating, and members to do this from original anime_ratings
popularAnimes = animeStats['rating']['size'] >= 100
animeStats[popularAnimes].sort_values([('rating','mean')], ascending=False).head(10)

rating          
                                     size      mean
name                                               
Kimi no Na wa.                       2199  8.297863
Ginga Eiyuu Densetsu                  903  8.239203
Steins;Gate                         19283  8.126796
Fullmetal Alchemist: Brotherhood    24574  8.028933
Gintama°                             1386  7.956710
Hunter x Hunter (2011)               8575  7.924082
Clannad: After Story                17854  7.835275
Monster                              4594  7.809099
Gintama                              4974  7.775231
Code Geass: Hangyaku no Lelouch R2  24242  7.765943

In [13]:
df = animeStats[popularAnimes].join(pd.DataFrame(similarAnimes, columns=['similarity']))
df.head()

/home/altock/anaconda3/envs/py35/lib/python3.5/site-packages/pandas/tools/merge.py:536: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,"(rating, size)","(rating, mean)",similarity
name,,,
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,782,5.774936,0.438522
&quot;Bungaku Shoujo&quot; Memoire,809,6.155748,0.430201
&quot;Bungaku Shoujo&quot; Movie,1535,6.457980,0.542192
.hack//G.U. Returner,730,4.804110,0.673044
.hack//G.U. Trilogy,1118,5.347943,0.648628


In [14]:
df.sort_values(['similarity'], ascending=False).head(10)

,"(rating, size)","(rating, mean)",similarity
name,,,
Dragon Ball Z,17032,6.772135,1.000000
Soratobu Yuureisen,163,5.380368,0.947088
Omae Umasou da na,171,6.824561,0.918776
Ginga Tetsudou Monogatari,144,5.638889,0.887691
Galaxy Angel 3 Specials,117,4.965812,0.865963
Dragon Ball,14117,6.597152,0.863344
Aquarion Movie: Ippatsu Gyakuten-hen,141,5.404255,0.861022
Sol Bianca,109,4.568807,0.856030
Ojamajo Doremi Na-i-sho,135,5.251852,0.854087


In [15]:
topDBZRelated = df.sort_values(['similarity'], ascending=False)[:10].index.values

In [16]:
for show in topDBZRelated:
    print(show, anime[anime.name == show].avg_rating.values)

Dragon Ball Z [ 8.32]
Soratobu Yuureisen [ 6.85]
Omae Umasou da na [ 8.13]
Ginga Tetsudou Monogatari [ 7.29]
Galaxy Angel 3 Specials [ 7.37]
Dragon Ball [ 8.16]
Aquarion Movie: Ippatsu Gyakuten-hen [ 6.93]
Sol Bianca [ 6.47]
Ojamajo Doremi Na-i-sho [ 7.77]
Youma [ 6.25]


In [17]:
animeRatings.head()

name,&quot;0&quot;,"&quot;Aesop&quot; no Ohanashi yori: Ushi to Kaeru, Yokubatta Inu",&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,&quot;Bungaku Shoujo&quot; Memoire,&quot;Bungaku Shoujo&quot; Movie,&quot;Eiji&quot;,.hack//G.U. Returner,.hack//G.U. Trilogy,.hack//G.U. Trilogy: Parody Mode,.hack//Gift,...,makemagic,"on-chan, Yume Power Daibouken!",s.CRY.ed,vivi,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki,◯
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN


In [18]:
# too slow
# corrMatrix = animeRatings.corr()
# corrMatrix.head()

In [19]:
# corrMatrix = corrMatrix.corr(method='pearson', min_periods=100)
# corrMatrix.head()

In [27]:
animeRatings["user_id"][1]

KeyError: 'user_id'

user_id,1,2,3,4,5,6,7,8,9,10,...,73507,73508,73509,73510,73511,73512,73513,73514,73515,73516
anime_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,10.0,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.0,NaN,NaN,NaN,NaN,NaN,8.0,NaN,10.0,NaN
6,NaN,NaN,NaN,-1.0,8.0,NaN,NaN,NaN,NaN,NaN,...,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
np.dot(userRatings[1], userRatings[2])

nan

In [10]:
user1 = np.ma.masked_array(userRatings[1],np.isnan(userRatings[1]))
user2 = np.ma.masked_array(userRatings[2], np.isnan(userRatings[2]))
np.dot(user1, user2)

nan

In [13]:
from scipy.sparse import csr_matrix
userMatrix = csr_matrix(userRatings.values)
print(userMatrix)

MemoryError: 